<a href="https://colab.research.google.com/github/urmzd/lepus-classifier/blob/main/notebooks/example_bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf /content/rabbit-classifier/
!git clone https://github.com/urmzd/rabbit-classifier
!pip install loguru pipetools pytorch-lightning

Cloning into 'rabbit-classifier'...
remote: Enumerating objects: 748, done.
remote: Counting objects: 100% (748/748), done.
remote: Compressing objects: 100% (445/445), done.
remote: Total 748 (delta 367), reused 520 (delta 205), pack-reused 0
Receiving objects: 100% (748/748), 51.62 MiB | 8.11 MiB/s, done.
Resolving deltas: 100% (367/367), done.
     |████████████████████████████████| 527 kB 4.2 MB/s 
     |████████████████████████████████| 134 kB 76.9 MB/s 
     |████████████████████████████████| 829 kB 81.7 MB/s 
     |████████████████████████████████| 398 kB 70.7 MB/s 
     |████████████████████████████████| 952 kB 89.1 MB/s 
     |████████████████████████████████| 596 kB 68.2 MB/s 
     |████████████████████████████████| 1.1 MB 79.2 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 271 kB 92.9 MB/s 
     |████████████████████████████████| 144 kB 88.8 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=4

/content


In [ ]:
import sys
SRC_PATH = "/content/rabbit-classifier/src"
sys.path.insert(0, SRC_PATH)

In [ ]:
from src.bootstrap import bootstrap